In [100]:
import pandas as pd
import requests
import bs4
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup
from pytrends.request import TrendReq
import numpy as np
import datetime
import time
import csv
import dateutil.relativedelta
import matplotlib.pyplot as plt; plt.rcdefaults()
from sklearn.linear_model import LinearRegression
from scipy import stats
import statsmodels.formula.api as sm

In [2]:
#static lists
#for knowyourmeme
uNames = []
Names = []
#for lifetime
emptyList = []
tooNoisy = []
newList = []

# **Scraping for Meme Names**

In [3]:
def uneditedNames(memes):
    for meme in memes:
        cats = meme.find_all("td")
        for cat in cats:
            if(cat.a != None):
                items = cat.a.attrs.keys()
                if "href" in items:
                    uNames.append(cat.a['href'])
                
def scrapeMemes(numRequests):
    
    headers = {'me':'Chrome/5.0'}  
    numRequests = numRequests #288
    
    for x in range(1,numRequests):
        print(x)
        time.sleep(1)
        page = ('https://knowyourmeme.com/memes/page/' + str(x))
        req = Request(page,headers=headers)
        webpage = urlopen(req).read()
        page_soup = soup(webpage, 'html.parser')
        memes = page_soup.body.find(id="content").find(id="maru").find(id="infinite-scroll-wrapper").find(id="entries").find(id="entries_list").tbody.find_all("tr")
        uneditedNames(memes)
        
    for name in uNames:
        if ("/memes/" in name):
            newName = name
            newName = newName.replace("/memes/","")
            if re.search(r'/?/',newName):
                ""
            else:
                newName = newName.replace("-"," ")
                newName = newName + " meme"
                Names.append(newName)

# Helper Functions

In [4]:
def totalNames(Names): #create ; DANGER: Will delete all files when ran
    with open('TotalNames', 'w') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(Names)

In [5]:
def checkNoise(x):
    try:
        return ((x.iloc[1].values[0] + x.iloc[2].values[0]) / 2) < 50
    except IndexError:
        return False
    

In [6]:
def finalDf(toIter, mainDf):
    for index, row in toIter.iterrows():
        temp = mainDf[index:]
        if checkNoise(temp):
            return temp
        
    return pd.DataFrame()

    

In [7]:
def halfLife(start,end):
    return end - start

In [8]:
def checkDups(tList,toCheck):
    nope = []
    yep = []
    for meme in tList:
        if meme in toCheck:
            yep.append(meme)
        else:
            nope.append(meme)
            
    print(len(nope))
    return nope

In [9]:
def calcDate(peak):
    time1 = peak - dateutil.relativedelta.relativedelta(months=6)
    time2 = peak + dateutil.relativedelta.relativedelta(months=6)
    time1 = time1.strftime("%Y-%m-%d ")
    time2 = time2.strftime("%Y-%m-%d")
    return time1 + time2

In [10]:
def to_date(x):
    return x[0].date()

In [106]:
def make_f_vals(df):
    fTime = []
    fPeak = []
    year = []
    newFP = []
    
    for index, row in df.iterrows():
        fTime.append(row['Half_Life'].total_seconds()/ datetime.timedelta(days=1).total_seconds())
        fPeak.append((row['Peak_Date']).toordinal())
        year.append(row['Peak_Date'].year)
    
    for x in fPeak:
        newFP.append(x-min(fPeak))
    
    df['fHalf_Life'] = fTime
    df['fPeak_Date'] = fPeak
    df['year'] = year
    final['Ordinal_Peak'] = newFP
    
    return df

# Finding lifetime of a meme

In [11]:
pytrends = TrendReq(hl='en-US', tz=360, retries=5, backoff_factor=0.1)

In [12]:
def runMemesL(memeList):
    counter = 0
    count = 0
    memeHalfLifes = pd.DataFrame(columns=['Peak_Date','Title','Half_Life'])
    for meme in memeList:
        try:
            kw_list = [meme]
            pytrends.build_payload(kw_list,  timeframe='all')
            memeInterest = pytrends.interest_over_time()
            count = count + 1
            counter = 0
            print(count)
            if memeInterest.empty:
                emptyList.append(meme)
            else:
                peakInterest = memeInterest[memeInterest[memeInterest.columns[0]] == 100]
                PeakInterestDf = memeInterest[peakInterest.index[0]:]
                halfInterest = PeakInterestDf[PeakInterestDf[memeInterest.columns[0]] < 50]
                finalInterest = finalDf(halfInterest, memeInterest)
                if finalInterest.empty:
                    tooNoisy.append(meme)
                else:
                    halfLifeVal = halfLife(peakInterest.iloc[0].name, finalInterest.iloc[0].name)
        
                    l = [[peakInterest.index[0]], memeInterest.columns[0], halfLifeVal]
                    temp = pd.DataFrame(data = [l], columns = ['Peak_Date','Title', 'Half_Life'])
                    memeHalfLifes = memeHalfLifes.append(temp)
        except:
            if counter > 3:
                return memeHalfLifes
            print(counter)
            print(meme)
            newList.append(meme)
            counter = counter + 1
            time.sleep(15)
            
    return memeHalfLifes

In [13]:
def runMemesD(memeFrame):
    counter = 0
    count = 0
    memeHalfLifes = pd.DataFrame(columns=['Peak_Date','Title','Half_Life'])
    for ir in memeFrame.itertuples():
        try:
            kw_list = [ir[2]]
            date = calcDate(ir[1])
            pytrends.build_payload(kw_list,  timeframe=date)
            memeInterest = pytrends.interest_over_time()
            count = count + 1
            counter = 0
            print(count)
            if memeInterest.empty:
                emptyList.append(ir[2])
            else:
                peakInterest = memeInterest[memeInterest[memeInterest.columns[0]] == 100]
                PeakInterestDf = memeInterest[peakInterest.index[0]:]
                halfInterest = PeakInterestDf[PeakInterestDf[memeInterest.columns[0]] < 50]
                finalInterest = finalDf(halfInterest, memeInterest)
                if finalInterest.empty:
                    tooNoisy.append(meme)
                else:
                    halfLifeVal = halfLife(peakInterest.iloc[0].name, finalInterest.iloc[0].name)
        
                    l = [[peakInterest.index[0]], memeInterest.columns[0], halfLifeVal]
                    temp = pd.DataFrame(data = [l], columns = ['Peak_Date','Title', 'Half_Life'])
                    memeHalfLifes = memeHalfLifes.append(temp)
        except Exception as e:
            print(e)
            if counter > 3:
                return memeHalfLifes
            print(counter)
            print(ir[2])
            counter = counter + 1
            time.sleep(15)
            
    return memeHalfLifes 

In [14]:
with open('NamesLeft', 'r') as f:
    reader = csv.reader(f)
    your_list = list(reader)
    tList = your_list[0]
    
memesFA = runMemesL(tList)

test = []
for x in memesFA['Peak_Date']:
    test.append(x[0].date())
memesFA['Peak_Date'] = test

#memesFA = runMemesL(['cat'])
df = pd.read_pickle('memes.pkl')
df = df.append(memesFA)

with open('NamesLeft', 'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(checkDups(checkDups(checkDups(tList,df['Title'].unique()),emptyList),tooNoisy))
      
with open('emptyList', 'r') as f:
    reader = csv.reader(f)
    your_list = list(reader)
    if len(your_list)>0:
        emptyListF = emptyList + your_list[0]
        
with open('emptyList', 'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(emptyListF)
        
with open('tooNoisy', 'r') as f:
    reader = csv.reader(f)
    your_listN = list(reader)
    if len(your_listN)>0:
        tooNoisy = tooNoisy + your_listN[0]
        
with open('tooNoisy', 'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(tooNoisy)
    
df.drop_duplicates(subset='Title', keep="first", inplace=True)

df = df.reset_index(drop=True)
df.to_pickle('memes.pkl')
print(len(df))  

1
2
2
0
0
1785


In [47]:
temp_df = pd.read_pickle('memes.pkl')

with open('curr_position.txt','r') as f:
    curr = int(f.read())

temp_memes = runMemesD(temp_df[curr:])

curr = curr + len(temp_memes)
print(curr)
with open('curr_position.txt','w') as f:
    f.write(str(curr))
    
test = []
for x in temp_memes['Peak_Date']:
    test.append(x[0].date())
temp_memes['Peak_Date'] = test

full_memes = pd.read_pickle('final_memes.pkl')
full_memes = full_memes.append(temp_memes)
full_memes.drop_duplicates(subset='Title', keep="first", inplace=True
full_memes = make_f_vals(full_memes)
full_memes = full_memes.reset_index(drop=True)
full_memes.to_pickle('final_memes.pkl')

1
2
3
4
5
name 'meme' is not defined
0
tsundere meme
6
7
8
9
10
11
12
13
name 'meme' is not defined
0
doomer boy meme
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
name 'meme' is not defined
0
eat hot chip and lie meme
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
name 'meme' is not defined
0
stonks meme
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
name 'meme' is not defined
0
wholesome memes meme
116
117
118
119
120
121
122
123
124
125
126
127
128
129
name 'meme' is not defined
0
stfu meme
130
131
132
133
name 'meme' is not defined
0
kill me meme
134
135
136
137
name 'meme' is not defined
0
wtf meme
138
139
140
141
142
143
144
145
name 'meme' is not defined
0
soon meme
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
name 'meme' is not defined
0
trap meme
1

In [103]:
stats.ttest_ind(final[final['year']==2019]['fHalf_Life'], final[final['year']==2018]['fHalf_Life'])

Ttest_indResult(statistic=2.1022436794538537, pvalue=0.0362521231910508)